[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/edwinRNDR/nlnml/blob/master/NB02_visualize_features.ipynb)


# CLIP prompt feature extractor

In [1]:
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 7.5 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86831 sha256=454e5326cbf46f15b4ad65026908c95e7f085e9988c0795bf9ff5cff3d77ef1e
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
Successfully built umap-learn


In [7]:
import umap
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from os import path
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler


# Connect to GoogleDrive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# This is where my dataset lives

In [4]:
dataset_location = '/content/gdrive/MyDrive/nln-dataset'

# Visualize features

In [ ]:
show_images = True

features_location = f"{dataset_location}/clip-features.csv"
dataset_location_normalized = f"{dataset_location}/normalized"

if not path.exists(features_location):
    print("Run NB01_extract_features first")
else:

  df = pd.read_csv(features_location)

  filter_labels = []

  if len(filter_labels) > 0:
      df = df[df["label"].isin(filter_labels)]


  features_df = df[df.columns[2:]]
  id_class_df = df[df.columns[0:2]]

  class_df = df[df.columns[1]]

  labels = sorted(set([path.dirname(x) for x in df["id"]]))


  features = StandardScaler().fit_transform(features_df)

  reducer = umap.UMAP()
  embedding_with_classes = reducer.fit_transform(features, y=class_df)
  embedding = reducer.fit_transform(features)
  embedding_df = pd.DataFrame( { "x": x, "y": y } )
  embedding_df = pd.concat([id_class_df, embedding_df], axis=1)

  embedding_df.to_csv(f"{dataset_location}/clip-embedding_{'_'.join(str(x) for x in filter_labels)}.csv", index=False)


  def get_image(path, zoom=1):
      return OffsetImage(plt.imread(f"{dataset_location_normalized}/{path}"), zoom=zoom*0.1)


  fig, ax = plt.subplots(1,1,figsize=(40,40))
  plt.scatter(*embedding.T, s=0.1, c=class_df, cmap='Spectral', alpha=1.0)

  if show_images:
    x,y = embedding.T
    for x0, y0, path in tqdm(zip(x, y, df["id"])):
      ab = AnnotationBbox(get_image(path), (x0, y0), frameon=False)
      ax.add_artist(ab)


  cbar = plt.colorbar(boundaries=np.arange(len(labels)+1)-0.5)
  cbar.set_ticks(np.arange(len(labels)))
  cbar.set_ticklabels(labels)
  plt.title("embedding")
  plt.show()


25it [00:14,  5.58it/s]